# Day 8, Part 1 - intro to ipyvolume
We'll start our journey into the 3RD DIMENSION with the package ```ipyvolume```

In [1]:
# if you don't get it:
#!pip install ipyvolume
# note: you may need:
#!jupyter nbextension enable --py --sys-prefix ipyvolume
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

# or you can do:
#!conda install -c conda-forge ipyvolume


import ipyvolume

Let's do a quick look at something with some random 3D data:

In [2]:
import numpy as np
x, y, z = np.random.random((3, 10000))
ipyvolume.quickscatter(x, y, z, size=1, marker="sphere")

Easy peasy!  Let's read in our simulation data and plot this!

In [3]:
from sys import path
path.append('../lesson02/')

from hermite_library import do_hermite

In [4]:
star_mass = 1.0 # stellar mass in Msun
planet_masses = np.array( [1.0, 0.05] ) # planet masses in Mjupiter

# [x,y,z] coords for each planet in AU
# NOTE: no z-coords!  These will be set to zero later on if you make them non-zero
planet_initial_position = np.array([ [5.0, 0.0, 6.0], 
                                     [10.0, 0.0, 3.0] ])

# planet's velocity at each position in km/s
# NOTE: no z-velocities!  These will be set to zero later on if you make them non-zero
planet_initial_velocity = np.array([ [0.0, 10.0, 1.0], 
                                     [0.0, -5.0, 0.0]])

In [5]:
# h is for hermite!
r_h, v_h, t_h, e_h = do_hermite(star_mass, 
                                planet_masses, 
                                planet_initial_position, 
                                planet_initial_velocity, 
                               tfinal=200, Nsteps=8800, 
                               threeDee=True)

In [6]:
# we'll have to reformat a bit for plotting
# right now, just all as one color
x = r_h[:,0,:].ravel()
y = r_h[:,1,:].ravel()
z = r_h[:,2,:].ravel()
ipyvolume.quickscatter(x, y, z, 
                       size=1, marker="sphere")

Let's make things a little more complicated and allow us to take a look at each orbit:

In [7]:
ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(r_h.shape[0]):
    ipyvolume.scatter(r_h[i,0,:],
                      r_h[i,1,:],
                      r_h[i,2,:], 
                      color=colors[i], 
                     marker='sphere')
ipyvolume.show()

So, this is pretty cool - we can now see how the orbits "precess" during their evolution and we can check out these shapes in 3D.

Note we can also plot more abstract spaces in 3D - like velocity space:

In [8]:
ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(v_h.shape[0]):
    ipyvolume.scatter(v_h[i,0,:],
                      v_h[i,1,:],
                      v_h[i,2,:], 
                      color=colors[i],
                     marker='sphere')
ipyvolume.show()

With this we can see how "jumpy" the velocity changes can get - this may be a numerical effect that is causing the precession of the orbits, or just how things are!

Ok, we can also show velocity by little vectors:

In [9]:
ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(v_h.shape[0]):
    ipyvolume.quiver(r_h[i,0,:],
                      r_h[i,1,:],
                      r_h[i,2,:],
                     v_h[i,0,:],
                      v_h[i,1,:],
                      v_h[i,2,:], 
                      color=colors[i])
ipyvolume.show()

So clearly the above is pointless - while it looks cool the arrows are too big and there are too many of them!  We can change this by taking "X" number of points:

In [10]:
step = 600
# also, length of arrays
N = v_h.shape[2]

ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(v_h.shape[0]):
    ipyvolume.quiver(r_h[i,0,0:N:step],
                      r_h[i,1,0:N:step],
                      r_h[i,2,0:N:step],
                     v_h[i,0,0:N:step],
                      v_h[i,1,0:N:step],
                      v_h[i,2,0:N:step], 
                      color=colors[i])
ipyvolume.show()

Now we can see a bit more about the motion - that their directions are opposite of eachother for example.  And that the central mass only moves slightly and around its center as well.

## Animation
Let's now figure out how to make an animation, and then save it for ourselves!  To do this, we'll need to format our data specifically as (time, position):

In [11]:
# for example, for particle 0:
r_h[:,0,:].T.shape

(8800, 3)

In [12]:
step = 10
# also, length of arrays
N = v_h.shape[2]

r = r_h[:,:,0:N:step]
v = v_h[:,:,0:N:step]

r_h.shape, r.shape, r[:,2,:].T.shape

((3, 3, 8800), (3, 3, 880), (880, 3))

In [13]:
# have to format color as well
#colors = np.empty((0,3))
color = [(1,0,0), (0,0,1), (0,1,0)]

#colors = np.array([])
colors = []
for i in range(r.shape[2]):
    colors.append(color)
    
colors = np.array(colors)

# order should be (times, points, colors)
colors = np.transpose(colors, (0, 2, 1)) # flip the last axes

colors.shape

(880, 3, 3)

In [14]:
ipyvolume.figure()

s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T, 
                      marker='sphere', 
                     color=colors)

ani = ipyvolume.animation_control(s, interval=200)

ipyvolume.show()

Note that we can only use the ```animation_control``` function on scatter plots or quiver plots, so we can't add lines or anything here.  Perhaps in a future release of ```ipyvolume```!

### Exercise
Try this with your own datasets!

Bonus: also try with animations of quiver plots

Bonus: is there anything else you want to animate? Should the size of the points change for example? (See ipyvolume docs for examples)

Bonus: do this with the galaxy simulations

# Part 2: ipyvolume + ipywidgets
Now let's combine the powers of widgets and ipyvolume to explore our datasets in 3D.

In [27]:
import ipywidgets

In [73]:
step = 100
# also, length of arrays
N = v_h.shape[2]

r = r_h[:,:,0:N:step]
v = v_h[:,:,0:N:step]

r[:,0,:].ravel().shape

(264,)

In [74]:
ipyvolume.figure()

# I'll have to think about this more -> didn't really work too well
#   the trick is to get them to "flatten" in the right way
# color trickery
#color = ['red', 'blue', 'green']
#colors = np.repeat(colors, r.shape[0])
#color = [(1,0,0), (0,0,1), (0,1,0)]
#colors = []
#for i in range(r.shape[2]):
#    colors.append(color)
#colors = np.array(colors).T
#colors = np.transpose(colors, (0, 2, 1)) # flip the last axes


x = r[:,0,:].ravel()
y = r[:,1,:].ravel()
z = r[:,2,:].ravel()

s = ipyvolume.scatter(x, y, z, 
                      marker='sphere')

#ipyvolume.show()
#colors.shape, r[:,0,:].shape

Now let's use widgets to change the size and color of our points:

In [75]:
size = ipywidgets.FloatSlider(min=0, max=30, step=0.1)
color = ipywidgets.ColorPicker()

Now we'll use a function we haven't used before from ipywidgets - something that links our scatter plot features to our widgets:

In [76]:
ipywidgets.jslink((s, 'size'), (size, 'value'))
ipywidgets.jslink((s, 'color'), (color, 'value'))

Link(source=(Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', line_material=ShaderMaterial(),…

Finally, well put all these things in a row: our plot, then our two linked widgets:

In [79]:
ipywidgets.VBox([ipyvolume.gcc(), size,  color])

### Exercise
Repeat this ipywidgets+ipyvolume for your own system.

Bonus: make different sliders for different planets to control size & color for each independently.

Bonus: make a quiver plot

Bonus: what other things can you think to add sliders/pickers for?  Hint: check out the docs for ```ipyvolume.quiver``` and ```ipyvolume.scatter``` to see what you can change.

# Part 3 - embedding

Finally, we might want to embed our creations on the web somewhere.  The first step is to make an ```html``` file from our in-python widgets.  Luckily, there is a function for that!

In [80]:
myVBox = ipywidgets.VBox([ipyvolume.gcc(), size,  color])

In [81]:
# if we don't do this, the bqplot will be really tiny in the standalone html
ipyvolume.embed.layout = myVBox.children[1].layout
ipyvolume.embed.layout.min_width = "400px"

In [86]:
ipyvolume.embed.embed_html("myPage.html", myVBox, offline=True, devmode=False)

In [84]:
!open myPage.html

### Exercise
Generate a page for your own simulation with all the controls you want!

**Bonus**: though we won't be covering it explicitly, you can actually deploy this to the web to be hosted on github pages.  The first thing you need to do is call ```embed``` a little differently:

In [88]:
ipyvolume.embed.embed_html("myPage.html", myVBox, offline=False, devmode=False)

Now, instead of opening it here, you need to add this file to your github page.  Again, we won't cover this in class, but feel free to ask for help after you've looked over the resources provided on today's course webpage under the "deploying to the web" header.

**Bonus**: add more linkage to your plot by linking to bqplot.  See the "Mixing ipyvolume with bqplot" example on the ```ipyvolume``` docs: https://ipyvolume.readthedocs.io/en/latest/bqplot.html#